In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file
from bokeh.palettes import Spectral7
from bokeh.io import output_notebook
#from graphviz import Digraph
import neuralNets
import numpy as np
import time
import copy
import os
import PIL
%matplotlib inline
plt.ion()
output_notebook()

Loading BokehJS ...

In [6]:
class NetBNOptim(nn.Module):
    def __init__(self):
        super(NetBNOptim, self).__init__()
        self.convIn = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(5,5), stride=(1,1), padding=(2,2))
        self.batNIn = nn.BatchNorm2d(num_features=32)
        self.reluIn = nn.ReLU()
        self.poolIn = nn.MaxPool2d(2, 2)
        
        self.convI2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(5,5), stride=(1,1), padding=(2,2))
        self.batNI2 = nn.BatchNorm2d(num_features=64)
        self.reluI2 = nn.ReLU()
        self.poolI2 = nn.MaxPool2d(2, 2)
        
        # Out CH = 32 !
        self.convI3 = nn.Conv2d(in_channels=64, out_channels=32, kernel_size=(5,5), stride=(1,1), padding=(2,2))
        self.batNI3 = nn.BatchNorm2d(num_features=32)
        self.reluI3 = nn.ReLU()
        self.poolI3 = nn.MaxPool2d(2, 2)
        
        self.outMul = int(setImageSize / 8) 
            
        self.fc = nn.Sequential(
            nn.Linear(32 * self.outMul * self.outMul, 1024),
            nn.ReLU(),
            nn.Dropout(p=0.5, inplace=True),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Dropout(p=0.5, inplace=True),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Dropout(p=0.5, inplace=True),
            nn.Linear(1024, len(datasetClasses))) # number of classes
        
        self.logsmax = nn.LogSoftmax()

    def forward(self, x):
        # Convolutional layer 1
        convInResult = self.convIn(x)
        convInResult.register_hook(save_grad('convInGrad'))
        x = self.poolIn(self.reluIn(self.batNIn(convInResult)))
        
        # Convolutional layer 2
        convI2Result = self.convI2(x)
        convI2Result.register_hook(save_grad('convI2Grad'))
        x = self.poolI2(self.reluI2(self.batNI2(convI2Result)))
        
        # Convolutional layer 3
        convI3Result = self.convI3(x)
        convI3Result.register_hook(save_grad('convI3Grad'))
        x = self.poolI3(self.reluI3(self.batNI3(convI3Result)))
        
        # Reshape the result for fully-connected layers
        x = x.view(-1, 32 * self.outMul * self.outMul)
        
        # Apply the result to fully-connected layers
        x = self.fc(x)
        
        # Finally apply the LogSoftMax for output
        x = self.logsmax(x)
        return x

In [7]:
netAviles = torch.load('AvilesCNN.net')

In [8]:
print(netAviles)

NetBNOptim (
  (convIn): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (batNIn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
  (reluIn): ReLU ()
  (poolIn): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (convI2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (batNI2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (reluI2): ReLU ()
  (poolI2): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (convI3): Conv2d(64, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (batNI3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
  (reluI3): ReLU ()
  (poolI3): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (fc): Sequential (
    (0): Linear (8192 -> 1024)
    (1): ReLU ()
    (2): Dropout (p = 0.5, inplace)
    (3): Linear (1024 -> 1024)
    (4): ReLU ()
    (5): Dropout (p = 0.5, inplace)
    (6): Linear (1024 -> 1024)
    (7): ReLU ()
    (8): Dropout (p = 0.5, inplace)
    (9): Linear (10